In [0]:
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import (
    LogisticRegressionModel, 
    RandomForestClassificationModel, 
    DecisionTreeClassificationModel, 
    GBTClassificationModel
)

# Load slicing model and test set
slicer_model = PipelineModel.load("/FileStore/models/slicer_top10")
test_ready   = spark.read.format("delta").load("/FileStore/data/test_ready")
test_topk    = slicer_model.transform(test_ready)

# Setup evaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="f1"
)

# Model paths
model_paths = {
    "Logistic Regression": "/FileStore/models/lr_top10_model_grid",
    "Random Forest": "/FileStore/models/rf_top10_model_grid",
    "Decision Tree": "/FileStore/models/dt_top10_model_grid",
    "GBT": "/FileStore/models/gbt_top10_model_grid"
}

# Corresponding model classes
model_classes = {
    "Logistic Regression": LogisticRegressionModel,
    "Random Forest": RandomForestClassificationModel,
    "Decision Tree": DecisionTreeClassificationModel,
    "GBT": GBTClassificationModel
}

# Store F1 scores
f1_scores = []

# Evaluate each model
for model_name, path in model_paths.items():
    print(f"\n🔍 Evaluating {model_name}...")

    model_class = model_classes[model_name]
    model = model_class.load(path)
    
    preds = model.transform(test_topk)
    f1 = evaluator.evaluate(preds)
    
    print(f"{model_name} - F1-score on test set: {f1:.4f}")
    
    f1_scores.append((model_name, f1))

# Sort and display results
f1_scores.sort(key=lambda x: x[1], reverse=True)

print("\n📊 Final model comparison (sorted by F1-score):")
for name, score in f1_scores:
    print(f"{name:20} ➤  F1-score: {score:.4f}")


F1-score no conjunto de teste: 0.9462
